<a href="https://colab.research.google.com/github/Yassmina-Abdo/realtime-head-pose-estimation/blob/main/Notebooks/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## Liberaries

In [1]:
import numpy as np
import pickle

import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, SVR,SVC
from sklearn.preprocessing import MinMaxScaler,StandardScaler
# from xgboost.sklearn import XGBRegressor
###########
import cv2
import mediapipe as mp
import numpy as np
import time
import pandas
import os
import cv2
import mediapipe as mp
import numpy as np
import time
from math import dist 
import pickle
from math import cos, sin
from sklearn.preprocessing import MinMaxScaler,StandardScaler


## Reading the prepared dataset and split it

In [2]:
path_local='../Dataset/Data/media_with_angles_low_normed.csv'
path_drive='/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/media_with_angles_low_normed.csv'
data =pd.read_csv(path_local)
data.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y463,y464,y465,y466,y467,y468,image_name,yaw,pitch,roll
0,-0.236824,-0.006193,-0.224011,-0.147134,0.032246,-0.006193,-0.159947,-1.133721,-0.159947,-0.121508,...,0.034117,-0.785904,-0.747465,-0.709027,-0.849968,-0.901219,image03195,0.725284,-0.140406,-0.174674
1,0.104239,-0.004250,0.056021,-0.076575,-0.016304,-0.016304,0.019859,-0.582855,0.007804,-0.004250,...,0.030482,-0.680720,-0.656611,-0.632502,-0.801262,-0.825371,image03247,-0.233745,-0.116700,0.095809
2,0.180066,-0.003883,0.180066,0.017758,-0.036345,-0.003883,0.115143,0.028579,0.115143,0.071861,...,0.047943,-0.525545,-0.482263,-0.460622,-0.633751,-0.698674,image03214,-1.330352,0.003072,0.362599
3,0.000114,-0.015663,0.031668,-0.110324,0.000114,0.000114,0.063222,-1.041159,0.078999,0.078999,...,-0.015938,-1.341195,-1.246533,-1.167649,-1.593625,-1.656732,image03238,-0.040907,-0.474087,0.019801
4,-0.107980,-0.005590,-0.056785,-0.069583,0.020008,0.032807,0.058404,-0.799111,0.084002,0.109599,...,0.043323,-0.852588,-0.814192,-0.775796,-0.916582,-0.954978,image03236,0.205732,-0.241359,-0.150232


In [3]:
X = data.iloc[:,:-4]
X

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y459,y460,y461,y462,y463,y464,y465,y466,y467,y468
0,-0.236824,-0.006193,-0.224011,-0.147134,0.032246,-0.006193,-0.159947,-1.133721,-0.159947,-0.121508,...,0.008491,-0.029947,0.008491,0.021304,0.034117,-0.785904,-0.747465,-0.709027,-0.849968,-0.901219
1,0.104239,-0.004250,0.056021,-0.076575,-0.016304,-0.016304,0.019859,-0.582855,0.007804,-0.004250,...,0.006374,-0.029789,-0.053898,0.018428,0.030482,-0.680720,-0.656611,-0.632502,-0.801262,-0.825371
2,0.180066,-0.003883,0.180066,0.017758,-0.036345,-0.003883,0.115143,0.028579,0.115143,0.071861,...,0.026302,-0.006159,0.047943,0.026302,0.047943,-0.525545,-0.482263,-0.460622,-0.633751,-0.698674
3,0.000114,-0.015663,0.031668,-0.110324,0.000114,0.000114,0.063222,-1.041159,0.078999,0.078999,...,-0.031715,-0.094822,-0.157930,-0.015938,-0.015938,-1.341195,-1.246533,-1.167649,-1.593625,-1.656732
4,-0.107980,-0.005590,-0.056785,-0.069583,0.020008,0.032807,0.058404,-0.799111,0.084002,0.109599,...,0.017726,-0.020671,0.030524,0.030524,0.043323,-0.852588,-0.814192,-0.775796,-0.916582,-0.954978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730,-0.184366,-0.006020,-0.115772,-0.033458,0.035136,0.048855,0.062574,-0.760561,0.117450,0.158607,...,0.023215,-0.004223,0.036934,0.036934,0.050653,-0.758764,-0.731326,-0.703888,-0.758764,-0.772483
1731,0.210580,-0.007949,0.168955,-0.007949,-0.028761,-0.018355,0.085706,-0.049574,0.085706,0.054488,...,0.031690,0.010878,0.052502,0.042096,0.062909,-0.509429,-0.488617,-0.467805,-0.582272,-0.634303
1732,-0.024386,-0.011763,-0.011763,-0.049632,0.000861,0.026107,0.063976,-0.642917,0.101845,0.114468,...,0.024959,-0.000287,-0.012910,0.037582,0.050205,-0.669311,-0.656688,-0.631442,-0.719803,-0.745050
1733,-0.221710,-0.000058,-0.166297,-0.110884,0.013796,-0.000058,-0.069324,-0.914374,-0.055471,-0.027764,...,0.039539,0.011832,0.094952,0.053392,0.067245,-0.597712,-0.570005,-0.542299,-0.556152,-0.597712


In [4]:
y = data.iloc[:,-3:]
y.head(1)

,yaw,pitch,roll
0,0.725284,-0.140406,-0.174674


## Scaling

In [5]:
# scaler = StandardScaler()
# scaler = scaler.fit(X.to_numpy())
# x_scaled= scaler.transform(X.to_numpy())

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), test_size=0.20, random_state=42, shuffle=True)

In [7]:
x_train

array([[ 8.15231046e-02, -1.19403207e-02,  1.41159717e-03, ...,
        -6.62243832e-01, -9.69337944e-01, -1.03609753e+00],
       [ 2.90918909e-02, -5.37726414e-03, -2.83567008e-02, ...,
        -4.76083607e-01, -6.36939664e-01, -6.82898537e-01],
       [-1.28295671e-02, -1.84375331e-04, -1.84375331e-04, ...,
        -7.91349709e-01, -9.30446819e-01, -9.55737202e-01],
       ...,
       [-1.03836858e-01, -7.43262253e-03, -7.62927909e-02, ...,
        -8.71564959e-01, -1.10568953e+00, -1.14700563e+00],
       [ 9.80113504e-03, -9.71277054e-03,  4.88289462e-02, ...,
        -3.07637483e-01, -3.17394436e-01, -3.75936153e-01],
       [-1.85544390e-01, -3.03650827e-03, -1.24708429e-01, ...,
        -1.02310226e+00, -1.34756072e+00, -1.40839668e+00]])

In [8]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 50):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width /2
        tdy = height /2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img


In [9]:
pkl_filename='model_svr.pkl'
with open(pkl_filename, 'rb') as file:
    pretrained_model = pickle.load(file)

In [10]:
pkl_filename='model_GB_yaw.pkl'
with open(pkl_filename, 'rb') as file:
    pretrained_GB_yaw_model = pickle.load(file)

In [11]:
pkl_filename='model_GB_pitch.pkl'
with open(pkl_filename, 'rb') as file:
    pretrained_GB_pitch_model = pickle.load(file)

In [12]:
pkl_filename='model_GB_roll.pkl'
with open(pkl_filename, 'rb') as file:
    pretrained_GB_roll_model = pickle.load(file)

## Try everything 

In [ ]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()

    start = time.time()

    # Flip the image horizontally for a later selfie-view display
    # Also convert the color space from BGR to RGB
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    # To improve performance
    image.flags.writeable = False
    
    # Get the result
    results = face_mesh.process(image)
    
    # To improve performance
    image.flags.writeable = True
    
    # Convert the color space from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = image.shape
    face_2d = []
    x_points=[]
    y_points=[]

    if results.multi_face_landmarks:   
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 1 :
                    nose_2d = (lm.x * img_w, lm.y * img_h)
                    nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)
                elif idx == 428 :
                    chin_2d = (lm.x * img_w, lm.y * img_h)
               
            
                x, y = int(lm.x * img_w), int(lm.y * img_h)
                x_points.append(x) 
                y_points.append(y) 

                face_2d.append([x, y])
                
#         Convert it to the NumPy array
        distX =  chin_2d[0] - nose_2d[0] 
        distY =  chin_2d[1] - nose_2d[1]
        distance = (distX**2 + distY**2)**0.5
        
        face_2d = np.array(face_2d, dtype=np.float64)
        x_point=np.array(np.transpose(x_points).astype('float'),dtype=np.float64).reshape(1,-1)
        y_point=np.array(np.transpose(y_points).astype('float'),dtype=np.float64).reshape(1,-1)
        
        x_point = (x_point-nose_2d[0])/distance
        y_point = (y_point-nose_2d[1])/distance
        new_data = np.concatenate((x_point,y_point),axis=1)
        
        # Load pretrained model
        
        angles = pretrained_model.predict(new_data) #yaw , pitch , roll
#         angle_yaw =pretrained_GB_yaw_model.predict(new_data)
#         angle_pitch =pretrained_GB_pitch_model.predict(new_data)
#         angle_roll =pretrained_GB_roll_model.predict(new_data)
# #         print(angles)
        x = angles[0][0] #yaw
        y = angles[0][1] #pitch
        z = angles[0][2] #roll
        
        
#         image = draw_axis(image,angle_pitch,-angle_yaw,-angle_roll,int(nose_2d[0]),int(nose_2d[1]))
        image = draw_axis(image,y,-x,-z,int(nose_2d[0]),int(nose_2d[1]))
        
        end = time.time()
        totalTime = end - start

        fps = 1 / totalTime
        print("FPS: ", fps)

        cv2.putText(image, f'FPS: {int(fps)}', (20,450), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

        mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec)


    cv2.imshow('Head Pose Estimation', image)

    if cv2.waitKey(5) & 0xFF == 27:
        break


cap.release()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(


FPS:  71.42523372443506


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  w

FPS:  99.62717339667458
FPS:  125.47653094803601
FPS:  71.35111594991835
FPS:  105.62602936362033
FPS:  96.52285175127722
FPS:  83.50030857438634


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  w

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  w

FPS:  48.080518140654554
FPS:  35.43623798178469
FPS:  35.1364138994069
FPS:  40.92003902439024


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  w

FPS:  42.53987443837033
FPS:  50.42746017433123
FPS:  45.59026086956522
FPS:  51.89364676770801
FPS:  53.008581358609796
FPS:  57.554771869639794
FPS:  72.63744523145662


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  w

In [ ]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# cap = cv2.VideoCapture(0)
path='D:/ITI Material/ML 1 Project/Head Pose Estimation ML Project-20220615T153744Z-001/Head Pose Estimation ML Project/Dataset/Images/image00053.jpg'

image = cv2.imread(path)

#     start = time.time()

# Flip the image horizontally for a later selfie-view display
# Also convert the color space from BGR to RGB
image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
cv2.imshow('Head Pose Estimation', image)



# # To improve performance
# image.flags.writeable = False

# # Get the result
# results = face_mesh.process(image)

# # To improve performance
# image.flags.writeable = True

# # Convert the color space from RGB to BGR
# image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

# img_h, img_w, img_c = image.shape
# face_2d = []
# x_points=[]
# y_points=[]

# if results.multi_face_landmarks:   
#     for face_landmarks in results.multi_face_landmarks:
#         for idx, lm in enumerate(face_landmarks.landmark):
#             if idx == 1 :
#                 nose_2d = (lm.x * img_w, lm.y * img_h)
#                 nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)
#             elif idx == 428 :
#                 chin_2d = (lm.x * img_w, lm.y * img_h)


#             x, y = int(lm.x * img_w), int(lm.y * img_h)
#             x_points.append(x) 
#             y_points.append(y) 

#             face_2d.append([x, y])

# #         Convert it to the NumPy array
#     distX =  chin_2d[0] - nose_2d[0] 
#     distY =  chin_2d[1] - nose_2d[1]
#     distance = (distX**2 + distY**2)**0.5

#     face_2d = np.array(face_2d, dtype=np.float64)
#     x_point=np.array(np.transpose(x_points).astype('float'),dtype=np.float64).reshape(1,-1)
#     y_point=np.array(np.transpose(y_points).astype('float'),dtype=np.float64).reshape(1,-1)

#     x_point = (x_point-nose_2d[0])/distance
#     y_point = (y_point-nose_2d[1])/distance
#     new_data = np.concatenate((x_point,y_point),axis=1)

#     # Load pretrained model

#     angles = pretrained_model.predict(new_data) #yaw , pitch , roll
# #         angle_yaw =pretrained_GB_yaw_model.predict(new_data)
# #         angle_pitch =pretrained_GB_pitch_model.predict(new_data)
# #         angle_roll =pretrained_GB_roll_model.predict(new_data)
# # #         print(angles)
#     x = angles[0][0] #yaw
#     y = angles[0][1] #pitch
#     z = angles[0][2] #roll


# #         image = draw_axis(image,angle_pitch,-angle_yaw,-angle_roll,int(nose_2d[0]),int(nose_2d[1]))
# image = draw_axis(image,y,-x,-z,int(nose_2d[0]),int(nose_2d[1]))

# mp_drawing.draw_landmarks(
#                 image=image,
#                 landmark_list=face_landmarks,
#                 connections=mp_face_mesh.FACEMESH_CONTOURS,
#                 landmark_drawing_spec=drawing_spec,
#                 connection_drawing_spec=drawing_spec)


# cv2.imshow('Head Pose Estimation', image)


